In [229]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [231]:
import numpy as np #대수학
import pandas as pd #전처리

import seaborn as sns #시각화
from matplotlib.patches import Patch
from matplotlib import pyplot as plt

from tensorflow.keras.models import load_model

plt.rcParams.update({'figure.max_open_warning': 0})
plt.style.use('fivethirtyeight')
cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm

import warnings
%matplotlib inline

import configparser
from sklearn.preprocessing import MinMaxScaler

import re
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import datetime as dt

import os # 경로

import numpy as np
import sys
import random as rn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import SimpleRNN, LSTM, Bidirectional

import configparser
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.layers import Dropout
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

In [211]:
pip install AutoReg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [212]:
# import required libraries

import xgboost as xgb
import pandas as pd
import os
import numpy as np
import requests
import datetime
import statsmodels.api as sm
from statsmodels.tsa.x13 import x13_arima_analysis, x13_arima_select_order
# from statsmodels.tsa.ar_model import AutoReg, ar_select_order
import copy
import IPython



import random as rn
import numpy as np #대수학
import pandas as pd #전처리
import seaborn as sns #시각화
from matplotlib.patches import Patch
from matplotlib import pyplot as plt

plt.rcParams.update({'figure.max_open_warning': 0})
plt.style.use('fivethirtyeight')
cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm

import warnings
get_ipython().run_line_magic('matplotlib', 'inline')

import configparser
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler

import re
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import datetime as dt

import os # 경로

import numpy as np
import sys
import random 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM, GRU

import configparser
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from statsmodels.multivariate.pca import PCA

ImportError: ignored

In [118]:
# Sets the working directory: the default directory is the one in which the variable list file is stored.

cwd = os.getcwd()
os.chdir(cwd)

In [119]:
# The function receives a response object when given `url'

def get_request_url(url):
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            print ("[%s]: Url Request Success" % datetime.datetime.now())
            return response.json()
        
    except Exception as e:
        print(e)
        print("[%s]: Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [120]:
# a function that collects a quarterly GDP growth series and future gdp growth

def get_actual_gdp_growth(key, start, end):
        
    stat_code = '200Y002'
    item_code1 = '10111'    
    
    url = f'http://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/{stat_code}/Q/{start}/{end}/{item_code1}'
    
    retData = get_request_url(url)
    
    if (retData == None):
        print("there is no data")
        return None
    else:
        
        data = retData['StatisticSearch']
        data = pd.DataFrame(data['row'])
        data = data[['DATA_VALUE', 'TIME']]        
        data.columns = ['GDP_GROWTH', 'TIME']
        
        data['DATE'] = pd.to_datetime(data['TIME']).dt.to_period('Q')
        data = data[['GDP_GROWTH', 'DATE']]    
        data = data.set_index('DATE')

        size = data.shape[0]
        
        gdp_series = data.resample('1M', convention = 'start').pad() 
        gdp_series.index = pd.PeriodIndex(gdp_series.index, freq = 'M')
        
        return gdp_series

In [122]:

# a function which collects the individual feature variable

def get_individual_var(access_key, start, end, stat_code, item_code1, item_code2, freq):
    
    key = access_key
    
    if item_code2 == 0:
        url = f'http://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/{stat_code}/{freq}/{start}/{end}/{item_code1}'
    else:
        url = f'http://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/{stat_code}/{freq}/{start}/{end}/{item_code1}/{item_code2}'
    
    retData = get_request_url(url)
    
    if (retData == None):
        print("there is no data")
        return None
    
    else:
        data = retData['StatisticSearch']
        data = pd.DataFrame(data['row'])
        return data 

In [127]:

# a function which obtains all monthly variables with additional information (lag, diff, group information)
# `start' and 'end' must be year-month format
# 'ispadding' refers to whether fill missing values for quarterly variable

def get_raw_data(key, start, end, IsPad):
    
    os.chdir("/content/drive/MyDrive/Colab Notebooks/gdp_re") '@'
    var_list = pd.read_csv('VarList_20220703_NABO.csv')
    num_var = var_list.shape[0]
    varname_set = var_list['VarName'].to_list()
    
    
    # a dictionary which contains the variable's lag structure, seasonality, difference
    
    dic = {}
    
    for i, j in zip(varname_set, range(len(varname_set))):
        dic[i] = {'LAG': var_list.iloc[j]['LAG'], 'DIFF': var_list.iloc[j]['DIFF'],
                  'ISA': var_list.iloc[j]['ISA'], 'GROUP': var_list.iloc[j]['GROUP']}
    
    
    
    # initialize data set: the set which consists of 1 variable
    stat_code = var_list.iloc[0]['STAT_CODE']
    item_code1 = var_list.iloc[0]['ITEM_CODE1']
    item_code2 = var_list.iloc[0]['ITEM_CODE2']
    lags = var_list.iloc[0]['LAG']
    freq = var_list.iloc[0]['FREQ']
    
    
    start_m = pd.to_datetime(start, format ='%Y%m')
    end_m = pd.to_datetime(end, format ='%Y%m')
          
    pi = pd.period_range(start_m, end_m, freq = "Q")
    period_list = list(pi.astype(str))
    
    start_q = period_list[0]
    end_q = period_list[-1]
    
    if freq == 'Q':
        data = get_individual_var(key, start_q, end_q, stat_code, item_code1, item_code2, freq) 
    else:
        data = get_individual_var(key, start, end, stat_code, item_code1, item_code2, freq) 
    
    data = data[['DATA_VALUE', 'TIME']]        
    data.columns = [varname_set[0], 'TIME']
    
    
    if freq == 'M':
        data['DATE'] = pd.to_datetime(data['TIME'], format = '%Y%m')
        data = data[[varname_set[0], 'DATE']]    
        data = data.set_index('DATE')
        data.index = pd.PeriodIndex(data.index, freq = 'M')
    
    else:
        data['DATE'] = pd.to_datetime(data['TIME']).dt.to_period('Q')
        data = data[[varname_set[0], 'DATE']]    
        data = data.set_index('DATE')
        
        if IsPad == 1:
                data = data.resample('1M', convention = 'start').pad()
        else:
                data = data.resample('1M', convention = 'end').last()
        
        data.index = pd.PeriodIndex(data.index, freq = 'M')
    
    
    # collect each variable through for-loop
    
    for i in range(1, num_var):
        
        stat_code = var_list.iloc[i]['STAT_CODE']
        item_code1 = var_list.iloc[i]['ITEM_CODE1']
        item_code2 = var_list.iloc[i]['ITEM_CODE2']
        freq = var_list.iloc[i]['FREQ']
        
        if freq == 'Q':
            var = get_individual_var(key, start_q, end_q, stat_code, item_code1, item_code2, freq) # var denotes the individual variable
        else:
            var = get_individual_var(key, start, end, stat_code, item_code1, item_code2, freq) 
            
        var = var[['DATA_VALUE', 'TIME']]        
        var.columns = [var_list.iloc[i]['VarName'], 'TIME']
  
        if freq == 'M':
            var['DATE'] = pd.to_datetime(var['TIME'], format = '%Y%m')
            var = var[[varname_set[i], 'DATE']]    
            var = var.set_index('DATE')
            var.index = pd.PeriodIndex(var.index, freq = 'M')
    
        else:
            var['DATE'] = pd.to_datetime(var['TIME']).dt.to_period('Q')
            var = var[[varname_set[i], 'DATE']]    
            var = var.set_index('DATE')
            
            if IsPad == 1:
                var = var.resample('1M', convention = 'start').pad()
            else:
                var = var.resample('1M', convention = 'end').last()
            
            var.index = pd.PeriodIndex(var.index, freq = 'M')
    
        data = data.join(var)
        
    return data, dic

In [128]:
raw_data, dic = get_raw_data('7MJBDXDXI03KBLP71JFE', '200001', '202112', 0)
raw_data.to_csv('raw_data.csv')

[2022-07-27 07:34:59.367315]: Url Request Success
[2022-07-27 07:35:00.858089]: Url Request Success
[2022-07-27 07:35:02.720892]: Url Request Success
[2022-07-27 07:35:04.303931]: Url Request Success
[2022-07-27 07:35:05.752945]: Url Request Success
[2022-07-27 07:35:07.191817]: Url Request Success
[2022-07-27 07:35:08.626312]: Url Request Success
[2022-07-27 07:35:10.176612]: Url Request Success
[2022-07-27 07:35:11.664761]: Url Request Success
[2022-07-27 07:35:13.050435]: Url Request Success
[2022-07-27 07:35:14.604293]: Url Request Success
[2022-07-27 07:35:15.969627]: Url Request Success
[2022-07-27 07:35:17.441351]: Url Request Success
[2022-07-27 07:35:18.790036]: Url Request Success
[2022-07-27 07:35:20.183795]: Url Request Success
[2022-07-27 07:35:21.575681]: Url Request Success
[2022-07-27 07:35:22.931308]: Url Request Success
[2022-07-27 07:35:24.344102]: Url Request Success
[2022-07-27 07:35:26.551344]: Url Request Success
[2022-07-27 07:35:28.017078]: Url Request Success


In [152]:

# a fuction which returns a vintage data and dictionary of other additional information

def get_vintage_data(start_vintage, end_vintage):
 
    os.chdir("/content/drive/MyDrive/Colab Notebooks/gdp_re")
    var_list = pd.read_csv('VarList_20220703_NABO.csv')
    varname_set = var_list['VarName'].to_list()
    
    # a dictionary which contains the variable's lag structure, seasonality, difference
    
    dic = {}
    
    for i, j in zip(varname_set, range(len(varname_set))):
        dic[i] = {'LAG': var_list.iloc[j]['LAG'], 'DIFF': var_list.iloc[j]['DIFF'],
                  'ISA': var_list.iloc[j]['ISA'], 'GROUP': var_list.iloc[j]['GROUP'],
                  'FREQ': var_list.iloc[j]['FREQ']}
    
    
    # reading the raw data and reflelct the vintage point
    
    raw_data =pd.read_csv('raw_data.csv')
    raw_data = raw_data.set_index('DATE')
    raw_data.index = pd.PeriodIndex(raw_data.index, freq = 'M')
    data = raw_data[(raw_data.index >= start_vintage) & (raw_data.index <= end_vintage)]
    
    # setting the vintage date points
    
    start_v = pd.to_datetime(start_vintage, format ='%Y%m')
    end_v = pd.to_datetime(end_vintage, format ='%Y%m')
    
    
    # get the month of the end_v
    current_month = end_v.month
    
    first_month = [1, 4, 7, 10]
    middle_month = [2, 5, 8, 11]
    
    
    # sets the current state of forecasting point either 초월, 중월, 말월
    
    state = 2
    
    if current_month in first_month:
        state = 0
    if current_month in middle_month:
        state = 1
    

    size = data.shape[0]
    data = data.astype(float)
   
    for i in varname_set:               
                    
                    num_lag = dic[i]['LAG'] 
                    var_freq = dic[i]['FREQ']
                    
                    if var_freq == 'M':
                        
                        if num_lag == 1:
                            
                            data[i][-1] = np.nan
                            
                        if num_lag== 2:
                            
                            for k in range(num_lag):
                                data[i][size - k - 1] = np.nan
                    
                    if var_freq == 'Q': # quarterly variable's lag is equal to 2 month
                        
                        
                        if state == 2:
                            
                            for k in range(num_lag):
                                data[i][size - k-1] = np.nan
                        else:
                            for k in range(num_lag*2):
                                data[i][size - k - 1] = np.nan
                    
                        
#                         # below is for padded data
                        
#                         if state == 2:
                            
#                             #for k in range(num_lag):
#                             for k in range(num_lag + 1): # for padded data      
#                                 data[i][size - k-1] = np.nan
                        
#                         elif state == 1:
#                             #for k in range(num_lag*2):
#                             for k in range(num_lag*2 + 1): # for padded data   
#                                 data[i][size - k - 1] = np.nan
                                
#                         else:
#                             #for k in range(num_lag*2):
#                             for k in range(num_lag*2 ): # for padded data   
#                                 data[i][size - k - 1] = np.nan        
                    
    data.index = pd.PeriodIndex(data.index, freq = 'M')
    
    return data, dic

In [153]:
# function that transforms into differenced series and seasonally-adjusted series

def transform(vintage_data, dic):
    
    # make a deep copy
    
    data = copy.deepcopy(vintage_data)
    
    # the set of variable names
    col_names = data.columns

    for var in col_names:
        
        # log difference or just difference
        if dic[var]['DIFF'] == 1:
            data[var] = np.log(data[var].astype(float)).diff()
        
        elif dic[var]['DIFF'] == 2:
            data[var] = data[var].astype(float).diff()
        
        else:
            pass

        
        # seasonal adjustment
        if dic[var]['ISA'] == 0:
            
            nan_check = data[var].isnull() 
            idx = data[var].index[nan_check == False]
            
            # checkpoints for where not-NaN value start and end in the time series
            start = np.where(data.index == idx[0])[0][0]
            end = np.where(data.index == idx[-1])[0][0]

            # generate seasnoally-adjusted variable
            
            orig_series = data[var].iloc[start:end+1]
            
            X13PATH = os.chdir("/content/drive/MyDrive/Colab Notebooks/gdp_re\\x13as")
            arima_res = x13_arima_analysis(endog = orig_series, maxorder = (2, 2), maxdiff = (1, 1), 
                                           x12path = X13PATH)

            # replace the original series with the adjusted variable
            data[var].iloc[start:end +1] = arima_res.seasadj
            
            # reset the currently working directory
            os.chdir("/content/drive/MyDrive/Colab Notebooks/gdp_re")
        else:
            pass
    
    
    return data

In [193]:
# a function that calculates the optimal number of factors through PCA: Kaiser Criterion

def get_factor_num(data):
    
    pc = PCA(data, standardize = False, missing = 'drop-row')
    ev = pc.eigenvals
    num_factor = np.sum([ev >= 1.0])
    
    return num_factor

In [194]:
# a function which returns the best AR model

def ar_model(y):               
        sel = ar_select_order(y, 12, old_names=False)
        res = sel.model.fit() 
        return res 


In [216]:
# a function which fills missing values with a dynamic factor model
# This model accounts for the complex structure of factor loadings: 
#`dic` arguments must be provided to classify each variable into a certain group that shares the same factor.
# In this model, one global factor, which influences all variables, is assumed. 

def df_model(data, dic, num_factor):
    
    cwd = os.getcwd()
    os.chdir(cwd)
    var_list = pd.read_csv('VarList_20220703_NABO.csv')
    
    num_q = (var_list['FREQ'] == 'Q').sum() # the number of quarterly variables
    
    varname_set = data.columns
    varname_set_q = varname_set[ : num_q].to_list()
    
    endog_q = data[varname_set_q].resample('Q').last()
    endog_q = endog_q.astype(float)
    
    #endog  = data.iloc[ :, num_q : ]
    
    endog  = data

    # factor-loadings structure: 
    
    my_factors = {}
    
    for i in varname_set:
        if dic[i]['GROUP'] != 'NOMINAL':
            my_factors[i] = ['GLOBAL', dic[i]['GROUP']]
        else:
            my_factors[i] = ['GLOBAL']
    
    #my_factors = {i:['GLOBAL', dic[i]['GROUP']] for i in varname_set}
    
    # setting up the Dynamic Factor Model
    
   # model = sm.tsa.DynamicFactorMQ(endog, endog_quarterly = endog_q, factors = num_factor, idiosyncratic_ar1 = True)
    
    model = sm.tsa.DynamicFactorMQ(endog, factors = num_factor, idiosyncratic_ar1 = True)
    
    # model = sm.tsa.DynamicFactorMQ(endog, endog_quarterly = endog_q)
    
    return model

In [217]:
# function which replace the missing values with the DFM


def fill_missing(data, dic):
    
    # NaN Check matrix:
    null_check = data.isnull() 
    
    # sets the opitmal number of factors in the data
    
    new_data = copy.deepcopy(data)
    
    print('Finding the optimal number of factor starts.')
    num_factor = get_factor_num(new_data)
    print('Finding the optimal number of factor ends.')
    
    # fits the DFM model
    
    my_df_model = df_model(data, dic, num_factor)
    res = my_df_model.fit()  
    dfm_forecasts = res.get_prediction(start = data.index[0], end = data.index[-1], information_set = 'smoothed')
        
    # replace missing values with forecasted values generated from the DFM model                                  
    
    null_check_binary = null_check.astype(int)
    filled_data = (1-null_check_binary).mul(np.nan_to_num(data))+(null_check_binary).mul(dfm_forecasts.predicted_mean)
    
    
    filled_data = filled_data[data.columns.to_list()]  
    
    return filled_data         

In [218]:

# a function which gets currently available informaton: 
# note: this function should be used when no forecasts are made on the missing values in the original dataset

def get_current_avail_x(data):
    
    X = data.iloc[:, 1:]
    col_names = X.columns
    
    new_x = np.zeros(X.shape[1])
    null_check = X.isnull() 
    
    
    for i in range(len(col_names)):

        idx = null_check[col_names[i]].index[null_check[col_names[i]] == False]
        end = np.where(X.index == idx[-1])[0][0]
        value = X[col_names[i]].iloc[end]
        new_x[i] = value
    
    return new_x   

In [219]:
# gets the network training

def train_network(X, y, lookback, mask_num):    
    
    seed_num = 1234
    np.random.seed(seed_num)
    rn.seed(seed_num)
    tf.random.set_seed(seed_num)
    
    

    model = keras.Sequential()
    model.add(keras.layers.InputLayer(input_shape= (lookback, X.shape[2])))
    model.add(keras.layers.Masking(mask_value= mask_num))
    model.add(keras.layers.LSTM(10, activation= 'sigmoid', return_sequences = False, 
                                kernel_regularizer = 'l1_l2', recurrent_regularizer = 'l1_l2'))        
    model.add(keras.layers.Dense(1))
    
    
#     inputs = keras.Input(shape=(lookback, X.shape[2]), dtype='float32')
#     masked = keras.layers.Masking(mask_value = mask_num)(inputs)
#     lstm = keras.layers.LSTM(300, activation='sigmoid')(masked)
#     output = keras.layers.Dense(1)(lstm)
#     model = keras.models.Model(inputs, output)
    
    opt = keras.optimizers.SGD(learning_rate= 0.01)
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 100, restore_best_weights= True)
    
    model.compile(optimizer = opt, loss = 'mse', metrics =[keras.metrics.RootMeanSquaredError()])
    model.fit(X,y, batch_size = 32, epochs = 2000, validation_split = 0.25, callbacks = [stop_early])
    return model


In [220]:

# a function which yields new instance of X at the foreascasting point
# this is an original function

def get_new_instance(start_m, end_m, lookback, dic, mask_num, step):
    
    data, dic = get_vintage_data(start_m, end_m)
    
    gdp_idx = data[data.iloc[:, 1].notnull()].index.tolist() # the list of index where gdp data is not null
    end_idx = data.index[-1] # end index of vintage dataset
    
    # determine the length of monthly forecasts
    
    if (end_idx.month == 1) | (end_idx.month == 4) | (end_idx.month == 7) | (end_idx.month == 10):
        
        period = pd.period_range(start = end_idx + 1, end = end_idx + 2 + 3*step, freq = 'M')
        
        # create NaN rows up to the end of the quarter
        for m in period:
            data = data.append(pd.Series(name = m)) 
    
    elif (end_idx.month == 2) | (end_idx.month == 5) | (end_idx.month == 8) | (end_idx.month == 11):
        
        period = pd.period_range(start = end_idx + 1, end = end_idx + 1 + 3*step, freq = 'M')
        
        # create NaN rows up to the end of the quarter
        for m in period:
            data = data.append(pd.Series(name = m)) 
    
    else:
        
        if step == 0:
            pass
        else:
            period = pd.period_range(start = end_idx + 1, end = end_idx + 3*step, freq = 'M')
            # create NaN rows up to the end of the quarter
            for m in period:
                data = data.append(pd.Series(name = m)) 
    
    
    # filling the missing values through DFM: 
    
    dataX = data.iloc[:, 1:]
    
    tr_data = transform(dataX, dic)
    filled_data = fill_missing(tr_data, dic)
      
    #filled_data.iloc[:, 1:4].loc[filled_data.iloc[:, 1:4].index > gdp_idx[-1]] = np.nan  # making unobserved gdp nan value
    
    
    scaler = StandardScaler()
    scaler.fit(filled_data)
    X = scaler.transform(filled_data)
    
#    X = X.to_numpy()
    
    new_X = create_lookback_data(X, lookback)
    new_X = np.array(new_X)
    new_inst = new_X[-1]
    
#     arr = []
    
#     for i in range(new_inst.shape[0]):
#         item = new_inst[i]
#         item = item[~np.isnan(item)]
#         arr.append(item)

#     arr = np.array(arr)
#     final_input = keras.preprocessing.sequence.pad_sequences(arr, maxlen = filled_data.shape[1], 
#                                                              padding ='pre', dtype = float, value = mask_num)
    
    return new_inst



In [221]:

# looked backed dataset: a function which returns lookback dataset and target variable

def get_training_data(data, target, lookback):
       
    X = []
    y = []
    
    for i in range(len(data)-lookback-1):
        t = []
        for j in range(1,lookback+1):
            # Gather past records upto the lookback period
            t.append(data[(i+j+1)])
        
        X.append(t)
        
        y.append(target[i+lookback+1])    
    
    return X, y

In [222]:
# create loolback dataset excep target variable

def create_lookback_data(data, lookback):
    
    X = []
    
    for i in range(len(data)-lookback-1):
        t = []
        for j in range(1,lookback+1):
            # Gather past records upto the lookback period
            t.append(data[(i+j+1)])
        
        X.append(t)
    return X


In [223]:

# state = 0 : first month
# state =1  : middle month
# state = 2: end month
# state = 3: all month


def gen_state_var(simul_data):
    
    simul_data['STATE'] = np.nan
    
    simul_data['STATE'].loc[(simul_data.index.month == 1) | (simul_data.index.month == 4) 
                   | (simul_data.index.month == 7) | (simul_data.index.month == 10)] = 0
      
    simul_data['STATE'].loc[(simul_data.index.month == 2) | (simul_data.index.month == 5) 
                   | (simul_data.index.month == 8) | (simul_data.index.month == 11)] = 1
    
    
    simul_data['STATE'].loc[(simul_data.index.month == 3) | (simul_data.index.month == 6) 
                   | (simul_data.index.month == 9) | (simul_data.index.month == 12)] = 2
    
    simul_data['STATE'] = simul_data['STATE'].astype(int)
    
    return simul_data


In [224]:


# compute RMSE metric
# 'start', end' must be in the form of 'y-m'. For example, '2015-01'
# state = 0 : first month
# state =1  : middle month
# state = 2: end month
# state = 3: all month

def rmse_cal(simul_data, start, end, state):
    
    data = gen_state_var(simul_data)
    
    if state != 3:
        data = data[data['STATE'] == state]
    else:
        pass

    data = data[(data.index <= end) & (data.index >= start) ]
    data = data.astype(float)
    
    fe = data['actual_y'] - data['forecasts'] # forecasting error
    fe2 = fe**2 # squared forecasting error
    val_rmse = np.sqrt(np.mean(fe2))
    
    return val_rmse
    

# compute MAE metric  

def mae_cal(simul_data, start, end, state):

    data = gen_state_var(simul_data)
    
    if state != 3:
        data = data[data['STATE'] == state]
    else:
        pass
 
    data = data[(data.index <= end) & (data.index >= start) ]
    data = data.astype(float)
    
    fe = data['actual_y']- data['forecasts'] # forecasting error
    fe_abs = np.absolute(fe) # absolute values of forecasting error
    val_mae = np.mean(fe_abs)
    
    return val_mae

In [225]:
def do_forecast_ex(start_q, end_q, start_m, end_m, start_m_training, key, lookback, IsRecursive): #'@step'
    
    start_q = start_q # starting quarter of the whole sample 
    end_q = end_q   # ending quarter of the whole sample
    key = key

    simul_data = get_actual_gdp_growth(key, start_q, end_q)
    simul_data.columns = ['actual_y']
    # simul_data.columns = ['actual_y', 'actual_y1', 'actual_y2', 'actual_y3']
    simul_data['forecasts'] = np.nan


    df_pre = simul_data[simul_data.index < start_m] 
    window = len(df_pre)


    df_post = simul_data[simul_data.index >= start_m]
    forecast_horizon = len(df_post)


    # out-of-sample period
    period = pd.period_range(start = start_m, end = end_m, freq = 'M')

    # training sample starting point
    start_index  = pd.Period(start_m_training, freq = 'M')

    start_time = datetime.datetime.now()

    for i in period:

        end_index = start_index + window

        start_m = str(start_index).replace('-', '')
        end_m = str(end_index).replace('-', '')

        # gets the vintage data at the forecasting time
        vintage_data, dic = get_vintage_data(start_m, end_m)

        idx = vintage_data[vintage_data['GDP'].notnull()].index.tolist() # index where gdp growth data is not null.

        y = vintage_data['GDP'] 

        dataX = vintage_data.iloc[:, 1:]


        # transforming the raw data and filling the missing values

        # print(f'[{dt.datetime.now()}]: Transforming the original data starts.')
        tr_data = transform(dataX, dic)
        # print(f'[{dt.datetime.now()}]: Transforming the original data ends.')

        # print(f'[{dt.datetime.now()}]: Filling the missing values starts.')
        filled_data = fill_missing(tr_data, dic)
        # print(f'[{dt.datetime.now()}]: Filling the missing values ends.')


    #     X = filled_data[filled_data.index <= idx[-1]] # features data

    #     scaler = StandardScaler()
    #     scaler.fit(X)
    #     X = scaler.transform(X)


    # #    X = X.to_numpy() # this need to be deleted when MinMaxScaler is used.


    #     y = y[y.index <= idx[-1]]


    #     # generate the lookbacked train data set with the corresponding label at the quarterly frequency:
        
    #     lookback = int(lookback)
        # mask_num = 0


        # train_X, train_y = get_training_data(X, y, lookback)
        # train_X = train_X[-1::-3] # this is for quarterly filtering
        # train_y = train_y[-1::-3] # this is for quarterly filtering
        # train_X = train_X[::-1] # reverse the sequence
        # train_y = train_y[::-1] # reverse the sequence

        # train_X = np.array(train_X)
        # train_X = train_X.reshape(train_X.shape[0], lookback, train_X.shape[2])

        # train_y = np.array(train_y)
        # train_y = train_y.reshape(train_y.shape[0], 1)

        # # gets the trained neural network with given data

        # print(f'[{dt.datetime.now()}]: Learning starts.')

In [226]:
def train_network(X, y, lookback, mask_num):    
    
    seed_num = 1234
    np.random.seed(seed_num)
    rn.seed(seed_num)
    tf.random.set_seed(seed_num)
    
    

    model = keras.Sequential()
    model.add(keras.layers.InputLayer(input_shape= (lookback, X.shape[2])))
    model.add(keras.layers.Masking(mask_value= mask_num))
    model.add(keras.layers.LSTM(10, activation= 'sigmoid', return_sequences = False, 
                                kernel_regularizer = 'l1_l2', recurrent_regularizer = 'l1_l2'))        
    model.add(keras.layers.Dense(1))
    
    
#     inputs = keras.Input(shape=(lookback, X.shape[2]), dtype='float32')
#     masked = keras.layers.Masking(mask_value = mask_num)(inputs)
#     lstm = keras.layers.LSTM(300, activation='sigmoid')(masked)
#     output = keras.layers.Dense(1)(lstm)
#     model = keras.models.Model(inputs, output)
    
    opt = keras.optimizers.SGD(learning_rate= 0.01)
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 100, restore_best_weights= True)
    
    model.compile(optimizer = opt, loss = 'mse', metrics =[keras.metrics.RootMeanSquaredError()])
    model.fit(X,y, batch_size = 32, epochs = 2000, validation_split = 0.25, callbacks = [stop_early])
    return model

In [227]:
start_q = '2001Q1' # whole sample starting point
end_q = '2021Q4'   # whole sample ending point
key = '7MJBDXDXI03KBLP71JFE'
start_m = '201501'
end_m = '202112'
start_m_training = '200101'
lookback = 10
IsRecursive = 1

forecasted_data = do_forecast_ex(start_q, end_q, start_m, end_m, start_m_training, key, lookback, IsRecursive)

[2022-07-27 08:25:11.417538]: Url Request Success
Finding the optimal number of factor starts.
Finding the optimal number of factor ends.


AttributeError: ignored

In [213]:
import statsmodels.api as sm
model = sm.tsa.DynamicFactorMQ()

AttributeError: ignored

In [179]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, None, 1)           48        
                                                                 
 lstm_20 (LSTM)              (None, None, 234)         220896    
                                                                 
 dropout_20 (Dropout)        (None, None, 234)         0         
                                                                 
 lstm_21 (LSTM)              (None, None, 117)         164736    
                                                                 
 dropout_21 (Dropout)        (None, None, 117)         0         
                                                                 
 lstm_22 (LSTM)              (None, None, 78)          61152     
                                                                 
 dropout_22 (Dropout)        (None, None, 78)         

In [99]:
model.layers

In [107]:
names = [weight.name for layer in model.layers for weight in layer.weights]
weights = model.get_weights()

# suppress scientific notation
np.set_printoptions(suppress=True) #If True, always print floating point numbers using fixed point notation
for name, weight in zip(names, weights):
    if name == 'embedding_3/embeddings:0': #@@
        kernel_0 = weight
    if name == 'lstm_12/lstm_cell_12/kernel:0':
        recurrent_kernel_0 = weight
    if name == 'lstm_12/lstm_cell_12/recurrent_kernel:0':
        bias_0 = weight
    if name == 'lstm_12/lstm_cell_12/bias:0':
        bias_0 = weight

        

    elif name == '@':
        output = weight

SyntaxError: ignored

15

In [58]:
names

['embedding_3/embeddings:0',
 'lstm_12/lstm_cell_12/kernel:0',
 'lstm_12/lstm_cell_12/recurrent_kernel:0',
 'lstm_12/lstm_cell_12/bias:0',
 'lstm_13/lstm_cell_13/kernel:0',
 'lstm_13/lstm_cell_13/recurrent_kernel:0',
 'lstm_13/lstm_cell_13/bias:0',
 'lstm_14/lstm_cell_14/kernel:0',
 'lstm_14/lstm_cell_14/recurrent_kernel:0',
 'lstm_14/lstm_cell_14/bias:0',
 'lstm_15/lstm_cell_15/kernel:0',
 'lstm_15/lstm_cell_15/recurrent_kernel:0',
 'lstm_15/lstm_cell_15/bias:0',
 'dense_3/kernel:0',
 'dense_3/bias:0']

In [53]:
len(model.layers)

10

In [36]:
def get_layer_output(layer_name, data):
    intermediate_layer_model = keras.Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
    
    return intermediate_layer_model.predict(data)  

In [52]:
outputs = {}
for layer in model.layers :
    outputs[layer.name] = get_layer_output(layer.name, X_train.iloc[0])
    print(layer.name, outputs[layer.name].shape)

embedding_3 (48, 1, 1)
lstm_12 (48, 1, 234)
dropout_12 (48, 1, 234)
lstm_13 (48, 1, 117)


dropout_13 (48, 1, 117)


lstm_14 (48, 1, 78)
dropout_14 (48, 1, 78)
lstm_15 (48, 10)
dropout_15 (48, 10)
dense_3 (48, 1)


In [14]:
def get_relavance(target_data, target_class) :
    e = get_layer_output('embedding_1', target_data).squeeze()
    
    #forword
    T = maxlen
    d = int(256/4)  # hidden units

    idx    = np.hstack((np.arange(0,d), np.arange(2*d,4*d))).astype(int) # indices of gates i,f,o together
    idx_i, idx_g, idx_f, idx_o = np.arange(0,d), np.arange(d,2*d), np.arange(2*d,3*d), np.arange(3*d,4*d) # indices of gates i,g,f,o separately

    h  = np.zeros((T,d))
    c  = np.zeros((T,d))

    gates_xh  = np.zeros((T, 4*d))  
    gates_hh  = np.zeros((T, 4*d)) 
    gates_pre = np.zeros((T, 4*d))  
    gates     = np.zeros((T, 4*d))  

    for t in range(T):
        gates_xh[t]     = np.dot(e[t], kernel_0)
        gates_hh[t]     = np.dot(h[t-1],recurrent_kernel_0)
        gates_pre[t]    = gates_xh[t] + gates_hh[t] + bias_0
        gates[t, idx]    = 1.0/(1.0 + np.exp(- gates_pre[t,idx]))
        gates[t,idx_g]  = np.tanh(gates_pre[t,idx_g]) 
        c[t]            = gates[t,idx_f]*c[t-1] + gates[t,idx_i]*gates[t,idx_g]
        h[t]            = gates[t,idx_o]*np.tanh(c[t])

    score = np.dot(h[t], output)    
    
    #backword

    C      = output.shape[1] # number of classes
    dx     = np.zeros(e.shape)

    dh          = np.zeros((T, d))
    dc          = np.zeros((T, d))
    dgates_pre  = np.zeros((T, 4*d))  # gates pre-activation
    dgates      = np.zeros((T, 4*d))  # gates activation

    ds                    = np.zeros((C))
    ds[target_class] = 1.0
    dy               = ds.copy()

    #맨처음을 0으로 시작하지 않게 위한조치
    dh[T-1]     = np.dot(dy, output.T)

    for t in reversed(range(T)): 
        dgates[t,idx_o]    = dh[t] * np.tanh(c[t])  # do[t]
        dc[t]             += dh[t] * gates[t,idx_o] * (1.-(np.tanh(c[t]))**2) # dc[t]
        dgates[t,idx_f]    = dc[t] * c[t-1]         # df[t]
        dc[t-1]            = dc[t] * gates[t,idx_f] # dc[t-1]
        dgates[t,idx_i]    = dc[t] * gates[t,idx_g] # di[t]
        dgates[t,idx_g]    = dc[t] * gates[t,idx_i] # dg[t]
        dgates_pre[t,idx]  = dgates[t,idx] * gates[t,idx] * (1.0 - gates[t,idx]) # d ifo pre[t]
        dgates_pre[t,idx_g]= dgates[t,idx_g] *  (1.-(gates[t,idx_g])**2) # d g pre[t]
        dh[t-1]            = np.dot(dgates_pre[t], recurrent_kernel_0.T)
        dx[t]           = np.dot(dgates_pre[t], kernel_0.T)
    
    return score, dx